# Phân loại chó và mèo

Trong bài này, ta sẽ xây dựng mô hình CNN để phân biệt ảnh của lớp chó và lớp mèo. Đầu tiên ta cần tải dataset từ Kaggle về sử dụng [link](https://www.kaggle.com/c/dogs-vs-cats/data). Chú ý cần phải đăng ký tài khoản trên trang kaggle để có thể tải dữ liệu. Sau khi tải về ta giải nén ra thư mục như sau:

    -dogs-vs-cats
        -train
            -cat.0.jpg
            -dog.0.jpg
        -test1
            -cat.1000.jpg
            -dog.1000.jpg

Khi giải quyết một vấn đề, điều đầu tiên ta nên phân loại vấn đề vô một nhóm, từ đó mới đưa ra cách giải quyết phù hợp. Bài toán này là bài toán phân loại ảnh, nghĩa là input là ảnh (ma trận), và output là class chó hoặc mèo, tức là phân loại nhị phân (binary classification). Có nhiều cách giải quyết vấn đề này, ví dụ như trải phẳng ảnh ra thành vector, rồi sử dụng MLP với lớp sau cùng là 2 neuron, cách này đã được phân tích trong phần bài giảng. 

Trong bài tập này, ta sẽ xây dựng mô hình CNN để giải quyết bài toán.

## Phân tích dữ liệu

Đối với dữ liệu hình ảnh, ta nên tìm cách trả lời những câu hỏi:
- Có bao nhiêu điểm dữ liệu cho mỗi class
- Mỗi điểm dữ liệu trông như thế nào, có gì đặc biệt
- Kích thước ảnh có quá chênh lệch không
- ...

Để trả lời những câu hỏi này, ta phải phân tích dữ liệu trước khi xây dựng mô hình, đây là bước rất quan trọng.

In [6]:
original_dataset_dir = 'D:\\datasets\\dogs-vs-cats\\train'
base_dir = 'D:\\datasets\\dogs-vs-cats\\preprocess'

train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

train_cats_dir = os.path.join(train_dir, 'cats')
train_dogs_dir = os.path.join(train_dir, 'dogs')

validation_cats_dir = os.path.join(validation_dir, 'cats')
validation_dogs_dir = os.path.join(validation_dir, 'dogs')

test_cats_dir = os.path.join(test_dir, 'cats')
test_dogs_dir = os.path.join(test_dir, 'dogs')

# Pytorch

## Xây dựng class cung cấp dữ liệu

In [10]:
from torchvision.io import read_image
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
import torchvision
import torchvision.models as models
from torchvision import transforms
import os
from PIL import Image

class CustomImageDataset(Dataset):
    def __init__(self, img_dir, transform=None, target_transform=None):
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        image = read_image(img_path)
        label = self.img_labels.iloc[idx, 1]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

In [11]:
batch_size=32
img_dimensions = 224

# Normalize to the ImageNet mean and standard deviation
# Could calculate it for the cats/dogs data set, but the ImageNet
# values give acceptable results here.
img_transforms = transforms.Compose([
    transforms.Resize((img_dimensions, img_dimensions)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225] )
    ])

img_test_transforms = transforms.Compose([
    transforms.Resize((img_dimensions,img_dimensions)),    
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225] )
    ])

def check_image(path):
    try:
        im = Image.open(path)
        return True
    except:
        print(path)
        return False

train_data_path = "D:\\datasets\\dogs-vs-cats\\preprocess\\train"
train_data = torchvision.datasets.ImageFolder(root=train_data_path,transform=img_transforms, is_valid_file=check_image)

validation_data_path = "D:\\datasets\\dogs-vs-cats\\preprocess\\validation"
validation_data = torchvision.datasets.ImageFolder(root=validation_data_path,transform=img_test_transforms, is_valid_file=check_image)

test_data_path = "D:\\datasets\\dogs-vs-cats\\preprocess\\test"
test_data = torchvision.datasets.ImageFolder(root=test_data_path,transform=img_test_transforms, is_valid_file=check_image)

num_workers = 6
train_data_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=num_workers)
validation_data_loader = torch.utils.data.DataLoader(validation_data, batch_size=batch_size, shuffle=False, num_workers=num_workers)
test_data_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=False, num_workers=num_workers)


if torch.cuda.is_available():
    device = torch.device("cuda") 
else:
    device = torch.device("cpu")

In [12]:
from tqdm import tqdm
import torch.nn.functional as F

def train(model, optimizer, loss_fn, train_loader, val_loader, epochs=5, device="cpu"):
    print(device)
    for epoch in range(epochs):
        training_loss = 0.0
        valid_loss = 0.0
        model.train()
        for batch in tqdm(train_loader):
            optimizer.zero_grad()
            inputs, targets = batch
            inputs = inputs.to(device)
            targets = targets.to(device)
            output = model(inputs)
            loss = loss_fn(output, targets)
            loss.backward()
            optimizer.step()
            training_loss += loss.data.item() * inputs.size(0)
        training_loss /= len(train_loader.dataset)
        
        model.eval()
        num_correct = 0 
        num_examples = 0
        for batch in tqdm(val_loader):
            inputs, targets = batch
            inputs = inputs.to(device)
            output = model(inputs)
            targets = targets.to(device)
            loss = loss_fn(output,targets) 
            valid_loss += loss.data.item() * inputs.size(0)
                        
            correct = torch.eq(torch.max(F.softmax(output, dim=1), dim=1)[1], targets).view(-1)
            num_correct += torch.sum(correct).item()
            num_examples += correct.shape[0]
        valid_loss /= len(val_loader.dataset)

        print('Epoch: {}, Training Loss: {:.4f}, Validation Loss: {:.4f}, accuracy = {:.4f}'.format(epoch, training_loss,
        valid_loss, num_correct / num_examples))

In [13]:
def test_model(model):
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_data_loader:
            images, labels = data[0].to(device), data[1].to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print('correct: {:d}  total: {:d}'.format(correct, total))
    print('accuracy = {:f}'.format(correct / total))

In [14]:
model_resnet18 = models.resnet18(pretrained=True)

# Freeze all params except the BatchNorm layers, as here they are trained to the
# mean and standard deviation of ImageNet and we may lose some signal
for name, param in model_resnet18.named_parameters():
    if("bn" not in name):
        param.requires_grad = False

# Replace the classifier
num_classes = 2

model_resnet18.fc = nn.Sequential(nn.Linear(model_resnet18.fc.in_features,512),
                                  nn.ReLU(),
                                  nn.Dropout(),
                                  nn.Linear(512, num_classes))

In [15]:
import torch.optim as optim

model_resnet18.to(device)
optimizer = optim.Adam(model_resnet18.parameters(), lr=0.001)
train(model_resnet18, optimizer, torch.nn.CrossEntropyLoss(), train_data_loader, validation_data_loader, epochs=2, device=device)

cuda


100%|██████████| 500/500 [00:38<00:00, 13.07it/s]


Epoch: 0, Training Loss: 0.0878, Validation Loss: 0.0660, accuracy = 0.9731


100%|██████████| 500/500 [00:30<00:00, 16.24it/s]


Epoch: 1, Training Loss: 0.0494, Validation Loss: 0.0328, accuracy = 0.9873


In [17]:
test_model(model_resnet18)

correct: 4435  total: 4500
accuracy = 0.985556


## Make predictions

In [20]:
import os
def find_classes(dir):
    classes = os.listdir(dir)
    classes.sort()
    class_to_idx = {classes[i]: i for i in range(len(classes))}
    return classes, class_to_idx

def make_prediction(model, filename):
    labels, _ = find_classes('D:\\datasets\\dogs-vs-cats\\preprocess/test')
    img = Image.open(filename)
    img = img_test_transforms(img)
    img = img.unsqueeze(0)
    prediction = model(img.to(device))
    prediction = prediction.argmax()
    print(labels[prediction])
    
make_prediction(model_resnet18, 'D:\\datasets\\dogs-vs-cats\\preprocess\\test/dogs/dog.11460.jpg')
make_prediction(model_resnet18, 'D:\\datasets\\dogs-vs-cats\\preprocess\\test/cats/cat.12262.jpg')

dogs
cats


In [ ]:
torch.save(model_resnet18.state_dict(), "./model_resnet18.pth")